In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
def calc(df):
    """
    Args:
        df: dataframe pandas contendo as informações do arquivo de tempo
    Returns:
        a média dos valores e o devio padrão
    """
    mean = df[0].mean()
    std = df[0].std()
    margem_de_erro = 1.96 * (std / np.sqrt(len(df[0])))
    
    return mean, margem_de_erro

In [ ]:
def plot_it(directory, output_name, x_label, all_devices=False):
    
    # data files
    if all_devices:
        plt.rcParams['figure.figsize'] = 12, 8
        
        files = [file for file in sorted(os.listdir(directory)) if not file.startswith('.')]
        labels = [x for x in files]
    else:
        files = [file for file in sorted(os.listdir(directory)) if file.startswith('complete')]
        labels = ["dispositivo-%s" % x[-2:] for x in files]
    
        if len(labels) == 4 or len(labels) == 8:
            plt.rcParams['figure.figsize'] = 12, 8
        if len(labels) == 12:
            plt.rcParams['figure.figsize'] = 16, 8
        if len(labels) == 16:
            plt.rcParams['figure.figsize'] = 22, 8
        if len(labels) == 20:
            plt.rcParams['figure.figsize'] = 28, 8
    
    for file in files:
        data = pd.read_csv("".join((directory, file)), header=None)
        mean, erro = calc(data)

        # todos os cenários
        if all_devices:
            plt.errorbar(labels.index(file) ,mean, yerr=erro, color='b', fmt='o', capsize=10)
            plt.xlabel(x_label)
        
        # um cenário
        else:
            plt.errorbar(labels.index("dispositivo-%s" % file[-2:]), mean, yerr=erro, color='b', fmt='o', capsize=10)
            plt.xlabel("Dispositivos")
    
    plt.xticks(np.arange(len(files)),labels)
    plt.ylabel("Tempo de execução (ms)")
    plt.savefig(output_name)
        
        

In [ ]:
plot_it("../dynamic_cloudlet/full_16/execs/",
          "/tmp/plot.png",
          "",
          False)

In [ ]:
# Plot dos erros

plt.rcParams['figure.figsize'] = 8, 6
df = pd.read_csv("experimentos_inicio_distinto/erros_totais", header=None)

# for i,j in zip(df[0], df[1]):
plt.bar(x=df[0], height=df[1])
plt.xticks(df[0])
plt.xlabel("Número de dispositivos")
plt.ylabel("Quantidade de erros no servidor")
plt.savefig("plots_inicio_distinto/erros")

In [ ]:
# Plot dos histogramas para dois servidores

from matplotlib.ticker import PercentFormatter

plt.rcParams['figure.figsize'] = 12, 8

server_1 = !cat experimentos_source-afis/umts/20_d_simultaneo-umts/mpos_server_android.log | grep "Current number of requests" | cut -f6 -d':'
# server_2 = !cat experimentos_2_servers_benchImage/Cartoonize/simultaneo/rede_umts/20-umts/server_2.log | grep "Current number of requests" | cut -f6 -d':'

percent_server_1 = pd.Series(server_1).value_counts(sort=False, normalize=True) * 100
# percent_server_2 = pd.Series(server_2).value_counts(sort=False, normalize=True) * 100
percent_server_1.index = percent_server_1.index.map(int)
# percent_server_2.index = percent_server_2.index.map(int)

df = pd.DataFrame()
df["Servidor 1"] = percent_server_1
# df["Servidor 2"] = percent_server_2
df.sort_index(inplace=True)

ax = df.plot(kind='bar')
ax.yaxis.set_major_formatter(PercentFormatter())

plt.xlabel("Requisições")
plt.ylabel("Execuções simultâneas")
plt.savefig("experimentos_source-afis/umts/plots-simultaneo/20_hist_3200_digitais")